# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [1]:
from viz import *

Widget Javascript not detected.  It may not be installed or enabled properly.


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='438ffe4f-0c9e-42b5-8220-8381e2aa035d', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='7b587411-e2f2-4adc-996b-85787033cb30', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='86853f37-795c-42dc-803c-20d2f7266612', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='aedd7030-8e61-4461-82a2-a84a46614b61', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='c0f986a8-15b5-490a-a009-e6be580d70e9', ...)


INFO:tensorflow:Restoring parameters from model_runs/human_4_eps1/classifymodel_1000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/human_4_eps1/classifymodel_1000.ckpt
[({'top': [1], 'right': [99], 'left': [1], 'bottom': [99]}, {'top': [1], 'right': [99], 'left': [1], 'bottom': [99]}), ({'top': [1], 'right': [None], 'left': [None], 'bottom': [1]}, {'top': [1], 'right': [None], 'left': [None], 'bottom': [1]}), ({'top': [None], 'right': [None], 'left': [None], 'bottom': [None]}, {'top': [None], 'right': [None], 'left': [None], 'bottom': [None]}), ({'top': [None], 'right': [None], 'left': [None], 'bottom': [None]}, {'top': [None], 'right': [None], 'left': [None], 'bottom': [None]}), ({'top': [None], 'right': [None], 'left': [None], 'bottom': [None]}, {'top': [None], 'right': [None], 'left': [None], 'bottom': [None]})]
data["label"]: 
[ 0.  0.  0.  0.  1.]
data["classifications"][i][0][0]: 
[  8.17713478e-14   3.80964622e-14   1.05536466e-30   7.60402408e-10
   1.00000000e+00]
